In [ ]:
# this is the Azure Notebook, to be run inside Azure hosted notebook.
import sys
!{sys.executable} -m pip install bokeh==1.4.0 --user

In [ ]:
import math
import pandas as pd
pd.options.display.html.table_schema = True
pd.options.display.max_rows = None
import bokeh.io
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.models.ranges import FactorRange
from bokeh.transform import factor_cmap
from bokeh.resources import INLINE
import bokeh.io

bokeh.io.reset_output()
bokeh.io.output_notebook(INLINE)

In [ ]:
import azure.cosmos
from azure.cosmos.partition_key import PartitionKey

database = cosmos_client.create_database_if_not_exists('actorperfdb')
print('Database actorperfdb created')

container = database.create_container_if_not_exists(id='actorperfsummary', partition_key=PartitionKey(path='/PartitionKey'))
print('Container actorperfsummary created')

In [ ]:
%%sql --database actorperfdb --container actorperfsummary --output df_cosmos
SELECT c.CommitId, c.TestName, c.TimeMean, c.TimeStdDev, c.MemoryMean, c.MemoryStdDev, c.CpuMean, c.CpuStdDev FROM c

In [ ]:
%%sql --database actorperfdb --container commitlog --output df_commits
SELECT c.Id, c.CommitId, c.Date, c.Author FROM c

In [ ]:
width = 800
height = 332

def plots_with_error_bars(title, data, color='#4080A0'):
    p = figure(plot_height=height, plot_width=width, title=title)
    
    df = data["mean"]
    xs = list(range(len(df.values)))
    ys = df.values
    yerrs = data["error"].values

    # create the coordinates for the errorbars             
    err_xs = []
    err_ys = []

    for x, y, yerr in zip(xs, ys, yerrs):
        err_xs.append((x, x))
        err_ys.append((y - yerr, y + yerr))

    # plot them
    p.multi_line(err_xs, err_ys, color=color)
    p.circle(xs, ys, color=color, size=5, line_alpha=0) 

    if p.y_range.start:
        p.y_range.start = min(p.y_range.start, math.floor(min(ys) / 10) * 10)
    else:
        p.y_range.start = math.floor(min(ys) / 10) * 10
    
    show(p)
        
def methods(obj):
    print('Methods:')
    print('\n'.join([x for x in dir(obj) if not x.startswith('_')]))
    

In [ ]:
# Use pandas DataFrame.join method to add the real commit date to each row so 
# we can then sort by date to get the correct column order in the graphs.
# Then we groupby the TestName so we can create one  plot per test.
df_commits.sort_values(['Date'], inplace=True)
joined = df_cosmos.set_index('CommitId').join(df_commits.set_index('CommitId'))
joined.sort_values(['Date'], inplace=True)
grouped =joined.groupby(['TestName'])
datasets = []
for g in grouped:
    test_name = g[0]
    test_data = g[1].rename(columns={'TimeMean':'mean', 'TimeStdDev':'error'})
    plots_with_error_bars(test_name, test_data)